In [50]:
!pip3 install torchvision

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 16.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   - -------------------------------------- 6.3/204.2 MB 29.7 MB/s eta 0:00:07
   -- ------------------------------------- 14.2/204.2 MB 35.6 MB/s eta 0:00:06
   ---- ----------------------------------- 22.5/204.2 MB 36.6 MB/s eta 0:00:05
   ------ --------------------------------- 30.9/204.2 MB 37.8 MB/s eta 0:00:05
   ------- -------------------------------- 39.6/204.2 MB 38.1 MB/s eta 0:00:05
   --------- ------------------------------ 47.4/204.2 MB 38.2 MB/s eta 0:00:05
   ---------- ----------------------------- 55.6/204.2 MB 38.5 MB/s eta 0:00:04
   ------------ --------------------------- 64.0/204.2 MB 38.8 MB/s eta 0:00:04
   -------------- ------------------------- 71.8/204.2 MB 38.8 MB/s eta 0:00:04
   --------------- ------------------------ 80.0/204.2 MB 39.0 M

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiagents4pharma 0.0.0 requires torch==2.2.2, but you have torch 2.6.0 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
import os
import cv2
import tifffile as tiff
import numpy as np
from mask2bbox import BBoxes
from pathlib import Path
import pandas as pd

# Define paths
base_path = r"C:\Users\Varsha\OneDrive\Documents\Github\cellType\cHL_2_MIBI"  # Change to your dataset path
output_folder = r"C:\Users\Varsha\OneDrive\Documents\Github\cellType\cHL_2_MIBI\cell_index.csv"  # Where to save cell crops
image_folders = ["1", "2", "3", "4", "5", "6"]  # Image folder names
output_dir = r"C:\Users\Varsha\OneDrive\Documents\Github\cellType\cHL_2_MIBI\cellCrops"

all_data=[]

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)
valid_boxes = 0
i=1
# Iterate over each image folder
for folder in image_folders:
    batch_crops = [] 
    img_dir = os.path.join(base_path, f"raw_image\{folder}")  # Adjust if needed
    seg_dir = os.path.join(base_path, f"segmentation\{folder}\H3_memSUM_noCD163_deepcell060_AutoHist_mpp1.75")
    print(seg_dir)
    # Get all channel images and segmentation masks
    img_paths = sorted(Path(img_dir).glob("*.tiff"))  # 46 channels per image
    mask_path = next(Path(seg_dir).glob("*segmentationMap.tif"), None)  # Single mask


    if mask_path is None:
        print(f"No segmentation mask found for folder {folder}, skipping.")
        continue

    # Load segmentation mask
    mask = tiff.imread(str(mask_path))
    # Get bounding boxes from segmentation mask
    all_boxes = BBoxes.from_mask(str(mask_path))
    print(len(all_boxes))
    # print(dir(all_boxes))
    # Remove the bounding boxes that are located on the edge of the image
    all_boxes = all_boxes.remove_from_edge()
    all_boxes = all_boxes.filter("sides",np.greater_equal,(5,5))
    print("after filtering: ",len(all_boxes))
    all_boxes = all_boxes.bboxes

    # Load all 46 channels as a single stacked array
    images = [tiff.imread(str(p)) for p in img_paths]
    image_stack = np.stack(images, axis=-1)  # Shape: (H, W, 46)
    print("image stack shape : ",image_stack.shape)
    
    # Iterate over detected cells
    for idx, bbox in enumerate(all_boxes):
        index, x1, y1, x2, y2 = map(int,bbox)  # Bounding box coordinates
        # Crop across all channels
        cell_crop = image_stack[y1:y2, x1:x2, :]  # Shape: (h, w, 46)
        if cell_crop.size == 0:
            continue

        valid_boxes += 1 
        # Resize to 224x224 for ResNet
        cell_crop_resized = cv2.resize(cell_crop, (224, 224))

        # Append the cropped image to the batch list
        batch_crops.append(cell_crop_resized)

        # Store metadata in CSV
        all_data.append([folder, i])
        i+=1

    # Save the batch of crops as a single .npy file
    if batch_crops:
        batch_crops = np.array(batch_crops)
        np.save(output_folder, batch_crops)
        print(f"Saved {batch_crops.shape[0]} cell crops to {output_folder}")

    print(f"Processed folder {folder}, extracted {len(all_boxes)} cells.")
    print("Valid crops obtained: ", valid_boxes)
    break


print("All npy: ",len(all_data))

print("Cell extraction complete!")


C:\Users\Varsha\OneDrive\Documents\Github\cellType\cHL_2_MIBI\segmentation\1\H3_memSUM_noCD163_deepcell060_AutoHist_mpp1.75
38112
after filtering:  36208
image stack shape :  (2048, 1536, 46)


SystemError: <built-in function resize> returned a result with an exception set

In [61]:
import os
import cv2
import tifffile as tiff
import numpy as np
from mask2bbox import BBoxes
import torch
from torchvision.ops import masks_to_boxes
from pathlib import Path

# Define paths
base_path = r"C:\Users\Varsha\OneDrive\Documents\Github\cellType\cHL_2_MIBI"  # Change to your dataset path
output_folder = r"C:\Users\Varsha\OneDrive\Documents\Github\cellType\cHL_2_MIBI\cellCrops"  # Where to save cell crops
image_folders = ["1", "2", "3", "4", "5", "6"]  # Image folder names

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Iterate over each image folder
for folder in image_folders:
    img_dir = os.path.join(base_path, f"raw_image\{folder}")  # Adjust if needed
    seg_dir = os.path.join(base_path, f"segmentation\{folder}\H3_memSUM_noCD163_deepcell060_AutoHist_mpp1.75")
    print(seg_dir)
    # Get all channel images and segmentation masks
    img_paths = sorted(Path(img_dir).glob("*.tiff"))  # 46 channels per image
    mask_path = next(Path(seg_dir).glob("*segmentationMap.tif"), None)  # Single mask


    if mask_path is None:
        print(f"No segmentation mask found for folder {folder}, skipping.")
        continue

    # Load segmentation mask
    mask = tiff.imread(str(mask_path))

    mask_tensor = torch.tensor(mask, dtype=torch.uint8)
    mask_tensor = (mask_tensor > 0).type(torch.uint8)
    mask_tensor = mask_tensor.unsqueeze(0)
    

    boxes = masks_to_boxes(mask_tensor)
    bboxes = boxes.tolist()

    # Load all 46 channels as a single stacked array
    images = [tiff.imread(str(p)) for p in img_paths]
    image_stack = np.stack(images, axis=-1)  # Shape: (H, W, 46)
    # Iterate over detected cells
    for i,bbox in enumerate(bboxes):
        x1, y1, x2, y2 = bbox # Bounding box coordinates
        print("here 1",x1,y1,x2,y2)
        x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
        # Crop across all channels
        cell_crop = image_stack[y1:y2, x1:x2, :]  # Shape: (h, w, 46)
        print(cell_crop)
        # Save as NumPy array
        crop_path = os.path.join(output_folder, f"{folder}_cell_{i}.npy")
        np.save(crop_path, cell_crop)
    

    print(f"Processed folder {folder}, extracted {len(bboxes)} cells.")
    break
print("Cell extraction complete!")


C:\Users\Varsha\OneDrive\Documents\Github\cellType\cHL_2_MIBI\segmentation\1\H3_memSUM_noCD163_deepcell060_AutoHist_mpp1.75
here 1 0.0 0.0 1535.0 2047.0
[[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.  